In [3]:
import sys 
import os

# this will hide the Blender log from the oputput
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

In [4]:
import bpy
from IPython.display import display, Video
from math import pi

# Delete all mesh objects from the scene
bpy.ops.object.select_all(action="DESELECT")
bpy.ops.object.select_by_type(type="MESH")
bpy.ops.object.delete()

# Add a torus
bpy.ops.mesh.primitive_torus_add(
    major_radius=1.5,
    minor_radius=0.75,
    major_segments=48 * 4,
    minor_segments=12 * 4,
    align="WORLD",
    location=(0, 1, 1),
)

# Assigning the torus to a variable
torus = bpy.context.active_object

# Create a new material and assign it to the torus
material = bpy.data.materials.new(name="RainbowGradient")
torus.data.materials.append(material)
material.use_nodes = True
nodes = material.node_tree.nodes

# Clear default nodes
for node in nodes:
    nodes.remove(node)

# Add a Gradient Texture and set it to a color ramp of a rainbow
gradient = nodes.new(type="ShaderNodeTexGradient")
gradient.gradient_type = "LINEAR"
gradient.location = (0, 0)

ramp = nodes.new(type="ShaderNodeValToRGB")
ramp.color_ramp.interpolation = "LINEAR"
ramp.location = (200, 0)
ramp.color_ramp.elements[0].color = (1, 0.5, 0, 1)
ramp.color_ramp.elements[1].color = (1, 0.3, 0, 1)


# Add Shader nodes
bsdf = nodes.new(type="ShaderNodeBsdfPrincipled")
bsdf.location = (400, 0)

output = nodes.new(type="ShaderNodeOutputMaterial")
output.location = (600, 0)

# Connect the nodes
material.node_tree.links.new(gradient.outputs["Color"], ramp.inputs[0])
material.node_tree.links.new(ramp.outputs["Color"], bsdf.inputs["Base Color"])
material.node_tree.links.new(bsdf.outputs["BSDF"], output.inputs["Surface"])


# Camera setup
camera = bpy.data.objects["Camera"]
camera.location = (5, -3, 4)
camera.data.dof.use_dof = True
camera.data.dof.focus_distance = 5
camera.data.dof.aperture_fstop = 4

# Render setup
path = "light_animation.mp4"
bpy.context.scene.render.fps = 30
bpy.context.scene.render.resolution_x = 1000
bpy.context.scene.render.resolution_y = 400
bpy.context.scene.render.image_settings.file_format = 'FFMPEG'
bpy.context.scene.render.ffmpeg.format = 'MPEG4'
bpy.context.scene.render.ffmpeg.codec = 'H264'
bpy.context.scene.render.ffmpeg.audio_codec = 'NONE'
bpy.context.scene.render.filepath = path
bpy.context.scene.frame_end = 90  # Adjust the end frame for 3 seconds



torus.rotation_euler = (0, 0, pi / 2)  

light = bpy.data.objects["Light"]
light.location = (0, 0, 2)
light.keyframe_insert(data_path="location", frame=1)
light.location = (5, 0, 2)
light.keyframe_insert(data_path="location", frame=60)

torus.keyframe_insert(data_path="rotation_euler", frame=30)
torus.rotation_euler[0] += pi / 2
torus.keyframe_insert(data_path="rotation_euler", frame=90)  

bpy.ops.render.render(animation=True)
display(Video(filename=path , embed=True , width=800))